<a href="https://colab.research.google.com/github/tsuli/LSC_project/blob/main/Task2_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# specify the data file name and url
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/'
datafile = url + 'UCI%20HAR%20Dataset.zip'

# download the zip file from the web server using curl
!curl $datafile --output UCI_HAR_Dataset.zip

# unzip the file
!unzip -qq UCI_HAR_Dataset.zip

# change the directory name to remove spaces
!mv -f UCI\ HAR\ Dataset UCI_HAR_DATASET

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.1M  100 58.1M    0     0  57.8M      0  0:00:01  0:00:01 --:--:-- 57.8M


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
# load the features and labels (subtract 1 as the labels aren't indexed from 0)
ytest = np.loadtxt('UCI_HAR_DATASET/test/y_test.txt').astype(np.float32) -1
ytrain = np.loadtxt('UCI_HAR_DATASET/train/y_train.txt').astype(np.float32) -1

# load the x,y,z body accelerations test data
xx=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_x_test.txt').astype(np.float32) 
yy=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_y_test.txt').astype(np.float32) 
zz=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_z_test.txt').astype(np.float32) 

# concatenate the arrays along the last dimension
xtest = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)
# (using None here adds an extra dimension of size 1 to the end of the array)

# follow the same approach for the train data
xx=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_x_train.txt').astype(np.float32) 
yy=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_y_train.txt').astype(np.float32) 
zz=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_z_train.txt').astype(np.float32) 

xtrain = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)

xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)
ytrain_enc = tf.keras.utils.to_categorical(ytrain)
yvalid_enc = tf.keras.utils.to_categorical(yvalid)
ytest_enc = tf.keras.utils.to_categorical(ytest)
print(xtrain.shape, ytrain_enc.shape, xvalid.shape, yvalid_enc.shape, xtest.shape, ytest_enc.shape)

(5881, 128, 3) (5881, 6) (1471, 128, 3) (1471, 6) (2947, 128, 3) (2947, 6)


In [ ]:
# input layer: xtrain, ytrain

conv_layer = tf.keras.layers.Conv1D(filters=32, kernel_size=(4),activation=tf.nn.relu,padding='same')
BatchNorm_layer = tf.keras.layers.BatchNormalization()
ReLu_layer = tf.keras.layers.ReLU()
pooling_layer = tf.keras.layers.AveragePooling1D()
dense_layer = tf.keras.layers.Dense(units=10,activation=tf.nn.softmax)


In [ ]:
def predict(x):
    x = conv_layer(x)
    x = BatchNorm_layer(x)
    x = ReLu_layer(x)
    x = pooling_layer(x)
    output_layer = dense_layer(x)
    
    return output_layer

In [ ]:
@tf.function
def loss(x,y):
    y_ = predict(x)
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_), axis=[1]))
    return cross_entropy

In [ ]:
lr = 0.0001
train_steps = 1000
# we'll use the Adam optimizer instead of gradient descent.
optimizer = tf.optimizers.Adam(lr)

In [ ]:
for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(xtrain,ytrain_enc)
        gradients = tape.gradient(current_loss, tape.watched_variables())
        optimizer.apply_gradients(zip(gradients, tape.watched_variables()))
        if i%100 == 0:
            print('Training Step:' + str(i) + ' Loss = ' + str(current_loss))

TypeError: ignored

In [ ]:
type(xtrain)

numpy.ndarray

In [ ]:
loss(xtrain)

<tf.Tensor: shape=(5881, 64, 7), dtype=float32, numpy=
array([[[0.1429282 , 0.14294979, 0.14269814, ..., 0.14267974,
         0.14280616, 0.1429682 ],
        [0.14295633, 0.14290585, 0.14274353, ..., 0.14275739,
         0.14278714, 0.14292538],
        [0.14284709, 0.1428807 , 0.14290573, ..., 0.1427768 ,
         0.14281234, 0.14292705],
        ...,
        [0.14321564, 0.14309932, 0.14237055, ..., 0.14261702,
         0.14269866, 0.14288303],
        [0.14304072, 0.14302532, 0.14263614, ..., 0.14274581,
         0.14271623, 0.1426857 ],
        [0.14283477, 0.1430172 , 0.14281273, ..., 0.14268729,
         0.14280511, 0.14263016]],

       [[0.14211537, 0.15021092, 0.13626702, ..., 0.134828  ,
         0.14219688, 0.135444  ],
        [0.14886558, 0.14664815, 0.1344754 , ..., 0.13806891,
         0.14013177, 0.13752423],
        [0.15077233, 0.15003264, 0.13011765, ..., 0.13428465,
         0.13516256, 0.14262545],
        ...,
        [0.14689939, 0.1546066 , 0.13232721, ..., 0.1